In [1]:
import copy, math
from collections import defaultdict

In [2]:
def get_next_node(open_set, heuristic_guess):
    v = None
    min_d = float('inf')
    for node in open_set:
        if node in heuristic_guess:
            guess = heuristic_guess[node]
            if guess < min_d:
                min_d = guess
                v = node
    return v

In [3]:
start = [
    [4,1,3],
    [0,2,5],
    [7,8,6]
]
# '4:1:3:0:2:5:7:8:6'
target = [
    [1,2,3],
    [4,5,6],
    [7,8,0]
]

In [4]:
def serialize(matrix):
    result = []
    for row in matrix:
        for col in row:
            result.append(str(col)) # [4,1,3,0,2,5,7,8,6]
    return ':'.join(result)
serialize(start)

'4:1:3:0:2:5:7:8:6'

In [5]:
def deserialize(state):
    splited = state.split(':') # ['4', '1', '3', '0'...]
    splited = [int(x) for x in splited]
    return [splited[:3], splited[3:6], splited[6:]]

serialized = serialize(start)
deserialize(serialized)

[[4, 1, 3], [0, 2, 5], [7, 8, 6]]

In [6]:
def get_neighbours(state):
    matrix = deserialize(state)
    blank_i, blank_j = -1, -1
    
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            if matrix[i][j] == 0:
                blank_i, blank_j = i, j
                break
    
    neighbours = []
    if blank_i > 0:
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i - 1][blank_j]
        new_matrix[blank_i - 1][blank_j] = 0
        neighbours.append(serialize(new_matrix))
    
    if blank_i < (n-1):
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i + 1][blank_j]
        new_matrix[blank_i + 1][blank_j] = 0
        neighbours.append(serialize(new_matrix))
    
    if blank_j > 0:
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i][blank_j - 1]
        new_matrix[blank_i][blank_j - 1] = 0
        neighbours.append(serialize(new_matrix))
    
    if blank_j < (n-1):
        new_matrix = copy.deepcopy(matrix)
        new_matrix[blank_i][blank_j] = new_matrix[blank_i][blank_j + 1]
        new_matrix[blank_i][blank_j + 1] = 0
        neighbours.append(serialize(new_matrix))
    
    return zip(neighbours, [1 for _ in neighbours])

In [7]:
def loyd_astar(start_node, target_node, h):
    open_set = set([start_node])
    
    parents = {}
    parents[start_node] = None
    
    cheapest_paths = defaultdict(lambda: float('inf'))
    cheapest_paths[start_node] = 0
    
    heuristic_guess = defaultdict(lambda: float('inf'))
    heuristic_guess[start_node] = h(start_node)
    
    path_found = False
    while len(open_set) > 0:
        current_node = get_next_node(open_set, heuristic_guess)
        
        if current_node == target_node:
            path_found = True
            break
        
        open_set.remove(current_node)
        for (neighbour_node, weight) in get_neighbours(current_node):
            new_cheapest_path = cheapest_paths[current_node] + weight
            
            if new_cheapest_path < cheapest_paths[neighbour_node]:
                parents[neighbour_node] = current_node
                cheapest_paths[neighbour_node] = new_cheapest_path
                heuristic_guess[neighbour_node] = new_cheapest_path + h(neighbour_node)
                
                if neighbour_node is not open_set:
                    open_set.add(neighbour_node)
        
    path = []
    if path_found:
        while target_node is not None:
            path.append(target_node)
            target_node = parents[target_node]
        path.reverse()
    
    return path

In [8]:
def loyd_h(state):
    state = deserialize(state)
    H = 0
    n = len(state)
    for i in range(n):
        for j in range(n):
            H += abs(state[i][j] % n - j) + abs(state[i][j] / n - i)    
    return H

In [9]:
path = loyd_astar(serialize(start), serialize(target), loyd_h)
path

['4:1:3:0:2:5:7:8:6',
 '0:1:3:4:2:5:7:8:6',
 '1:0:3:4:2:5:7:8:6',
 '1:2:3:4:0:5:7:8:6',
 '1:2:3:4:5:0:7:8:6',
 '1:2:3:4:5:6:7:8:0']

In [10]:
for step in path:
    print(deserialize(step))

[[4, 1, 3], [0, 2, 5], [7, 8, 6]]
[[0, 1, 3], [4, 2, 5], [7, 8, 6]]
[[1, 0, 3], [4, 2, 5], [7, 8, 6]]
[[1, 2, 3], [4, 0, 5], [7, 8, 6]]
[[1, 2, 3], [4, 5, 0], [7, 8, 6]]
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]


Uopstenje heuristike

In [11]:
import math
def serialize(matrix):
    values = []
    for row in matrix:
        for col in row:
            values.append(str(col))
    return ':'.join(values)

def deserialize(value):
    splited = [int(x) for x in value.split(':')]
    n = int(math.sqrt(len(splited)))
    result = []
    for i in range(n):
        result.append([x for x in splited[i*n:(i+1)*n]])
    return result

In [12]:
def find_in_matrix(value, matrix):
    n = len(matrix)
    m = len(matrix[0])
    for i in range(n):
        for j in range(m):
            if value == matrix[i][j]:
                return i, j
    return None, None


def manhattan_sum(current_node, target_node):
    '''
        Suma menhetn rastojanja izmedju elemenata iz current_matrix i target_matrix
           1|3|2         1|2|3
        A= 4|6|5     B=  4|7|5
           7|8|0         0|8|6
        
        Suma menhetn rastojanja za elemente iz prve matrice u odnosu na elemente iz druge matrice
        H = d(A,B,1) + d(A,B,3) + d(A,B,2) + d(A,B,4) + d(A,B,6)...
        H = 0 + 1 + 1 + 0 + 2 + 0 + 2 + 0 + 2
    '''
    A = deserialize(current_node)
    B = deserialize(target_node)
    n = len(A)
    H = 0
    for i in range(n):
        for j in range(n):
            value = A[i][j]
            target_i, target_j = find_in_matrix(value, B)
            H += abs(i - target_i) + abs(j - target_j)
    return H
def chebisev():
    pass
def euclid_sum(current_node, target_node):
    A = deserialize(current_node)
    B = deserialize(target_node)
    n = len(A)
    H = 0
    for i in range(n):
        for j in range(n):
            value = A[i][j]
            target_i, target_j = find_in_matrix(value, B)
            H += math.sqrt(pow((i - target_i),2) + pow(j - target_j,2))
    return H

In [13]:
def astar_loyd(start_node, target_node, h):
    open_set = set([start_node])
    
    parents = {}
    parents[start_node] = None
    
    cheapest_path = defaultdict(lambda: float('inf'))
    cheapest_path[start_node] = 0
    
    heuristic_guess = defaultdict(lambda: float('inf'))
    heuristic_guess[start_node] = h(start_node, target_node) # <-------!
    
    path_found = False
    while len(open_set) > 0:
        current_node = get_next_node(open_set, heuristic_guess)
        
        if current_node == target_node:
            path_found = True
            break
        
        
        open_set.remove(current_node)
        for neighbour_node, weight in get_neighbours(current_node):
            new_cheapest_path = cheapest_path[current_node] + weight
            if new_cheapest_path < cheapest_path[neighbour_node]:
                parents[neighbour_node] = current_node
                cheapest_path[neighbour_node] = new_cheapest_path
                heuristic_guess[neighbour_node] = new_cheapest_path + h(neighbour_node, target_node) # <-----!
                if neighbour_node not in open_set:
                    open_set.add(neighbour_node)

    path = []
    if path_found:
        while target_node is not None:
            path.append(target_node)
            target_node = parents[target_node]
        path.reverse()
    return path

In [14]:
def astar_loyd_test(start_node, target_node, h):
    ######
    
    pass
    ######

In [15]:
start = [
    [2,0,3],
    [1,4,5],
    [7,8,6]
]
target = [
    [1,2,3],
    [4,5,6],
    [7,8,0]
]
path = astar_loyd(serialize(start), serialize(target), manhattan_sum)
path

['2:0:3:1:4:5:7:8:6',
 '0:2:3:1:4:5:7:8:6',
 '1:2:3:0:4:5:7:8:6',
 '1:2:3:4:0:5:7:8:6',
 '1:2:3:4:5:0:7:8:6',
 '1:2:3:4:5:6:7:8:0']

In [16]:
['2:0:3:1:4:5:7:8:6',
 '0:2:3:1:4:5:7:8:6',
 '1:2:3:0:4:5:7:8:6',
 '1:2:3:4:0:5:7:8:6',
 '1:2:3:4:5:0:7:8:6',
 '1:2:3:4:5:6:7:8:0']

['2:0:3:1:4:5:7:8:6',
 '0:2:3:1:4:5:7:8:6',
 '1:2:3:0:4:5:7:8:6',
 '1:2:3:4:0:5:7:8:6',
 '1:2:3:4:5:0:7:8:6',
 '1:2:3:4:5:6:7:8:0']